In [6]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import joblib
import uuid
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns


from src.utils.feature_cleaner import clean_features
from src.utils.feature_selector import select_features
from src.models.matching_model import get_model
from src.utils.trial_split import prepare_splits
from src.train.model_configs import MODEL_CONFIGS, CLASSIC_MODEL_CONFIGS
from src.utils.label_maps import TOPIC_TO_META
from src.utils.balance_topic_trials import balance_trials_by_topic

from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# ───────────────────────────────────────────────────────────── #
# Full Notebook Code: EEG → Topic Classification (MLP + Classic)
# ───────────────────────────────────────────────────────────── #

# ========== CONFIG ==========
PREFIX = "/absolute/or/relative/project_root/"   # <- change & keep trailing slash
DATA_PATH = f"{PREFIX}data/clean_full_dataset_with_avg_epochs.csv"
LABEL_PATH = f"{PREFIX}data/merged_embedded_reports.csv"
SAVE_DIR = Path(f"{PREFIX}notebooks/demo_outputs/")
SAVE_DIR.mkdir(parents=True, exist_ok=True)
LABEL_ENCODER_PATH = f"{PREFIX}notebooks/demo_outputs/topic_label_encoder.pkl"
SCALER_SAVE_PATH = f"{PREFIX}notebooks/demo_outputs/matching_scaler.pkl"


MODEL_TYPE = "lightgbm_tuned"  # or "mlp", rf_default, xgb_default ...
FEATURE_GROUPS = ["rel_theta"] # EEG Feature groups
LABEL_COLUMN = "topic_label"
EPOCHS = 100
BATCH_SIZE = 32
SEED = 42
EPOCH = 1 # or avg01, [1,2] ...

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(SEED)
np.random.seed(SEED)

# ========== LOAD & CLEAN ==========
eeg_df = pd.read_csv(DATA_PATH)
label_df = pd.read_csv(LABEL_PATH)
label_df["topic_label"] = label_df["Meta Topic ID"].map(TOPIC_TO_META)
label_df["trial_name"] = label_df["trial_name"].str.replace(".edf", "", regex=False)

eeg_df = clean_features(eeg_df)

if isinstance(EPOCH, str) and EPOCH.startswith("avg"):
    eeg_df = eeg_df[eeg_df["epoch"] == EPOCH]
else:
    eeg_df["epoch"] = pd.to_numeric(eeg_df["epoch"], errors="coerce")
    eeg_df = eeg_df[eeg_df["epoch"].isin([EPOCH])]

eeg_df = eeg_df.reset_index(drop=True)
eeg_df = eeg_df.merge(label_df[["trial_name", "Meta Topic ID"]], on="trial_name", how="left")
eeg_df["topic_label"] = eeg_df["Meta Topic ID"].map(TOPIC_TO_META)

# ========== SPLIT ==========
train_df_full, val_folds, test_df = prepare_splits(eeg_df, trial_column="trial_name", test_size=0.1, seed=SEED)

print(f"Total unique trials: {eeg_df['trial_name'].nunique()}")
print(f"Train+Val trials: {train_df_full['trial_name'].nunique()}")
print(f"Test trials: {test_df['trial_name'].nunique()}")

test_trial_names = test_df["trial_name"].unique()
with open(SAVE_DIR / "test_trials.txt", "w") as f:
    for trial in test_trial_names:
        f.write(trial + "\n")

results = []
experiment_ids = []
base_experiment_id = str(uuid.uuid4())

# ========== CROSS-VALIDATION LOOP ==========
for fold_idx, val_df in enumerate(val_folds):
    print(f"\n===== Fold {fold_idx + 1} =====")
    experiment_id = f"{base_experiment_id}_fold{fold_idx + 1}"
    experiment_ids.append(experiment_id)

    val_trial_names = val_df["trial_name"].unique()
    with open(SAVE_DIR / f"val_trials_fold_{fold_idx + 1}.txt", "w") as f:
        for trial in val_trial_names:
            f.write(trial + "\n")

    train_df = train_df_full[~train_df_full["trial_name"].isin(val_trial_names)].reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)

    train_df = balance_trials_by_topic(train_df, topic_col=LABEL_COLUMN)

    # Encode labels
    label_encoder = LabelEncoder()
    label_encoder.fit(train_df[LABEL_COLUMN])

    # Select features
    X_train, y_train = select_features(train_df, FEATURE_GROUPS, None, LABEL_COLUMN, None)
    X_val, y_val = select_features(val_df, FEATURE_GROUPS, None, LABEL_COLUMN, None)
    y_train_encoded = label_encoder.transform(y_train)
    y_val_encoded = label_encoder.transform(y_val)

    # Scale
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.fit_transform(X_val)

    # Save encoder + scaler
    joblib.dump(label_encoder, LABEL_ENCODER_PATH)
    joblib.dump(scaler, SCALER_SAVE_PATH)

    if MODEL_TYPE == "mlp":
        model_config = MODEL_CONFIGS["simple_512"]
        model = get_model("mlp", model_config, input_dim=X_train.shape[1], output_dim=len(label_encoder.classes_)).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=1e-3)

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val_encoded, dtype=torch.long)

        train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=BATCH_SIZE, shuffle=True)

        for epoch in range(EPOCHS):
            model.train()
            train_loss = 0.0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            print(f"Epoch {epoch + 1}: Loss = {train_loss / len(train_loader):.4f}")

        # Save model
        torch.save(model.state_dict(), SAVE_DIR / f"model_fold{fold_idx + 1}.pt")

        model.eval()
        with torch.no_grad():
            preds_val = model(X_val_tensor.to(device)).argmax(dim=1).cpu().numpy()
    else:
        config = CLASSIC_MODEL_CONFIGS[MODEL_TYPE]
        model = get_model(model_type=config["model_type"], model_config=config, task_type="classification")
        model.fit(X_train, y_train_encoded)

        # Save model
        joblib.dump(model, SAVE_DIR / f"model_fold{fold_idx + 1}.pt")

        preds_val = model.predict(X_val)

    acc = accuracy_score(y_val_encoded, preds_val)
    f1 = f1_score(y_val_encoded, preds_val, average="weighted")
    cm = confusion_matrix(y_val_encoded, preds_val)

    unique_labels = np.unique(np.concatenate([y_val_encoded, preds_val]))
    class_names = label_encoder.inverse_transform(unique_labels)

    cm = confusion_matrix(y_val_encoded, preds_val, labels=unique_labels)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=class_names,
                yticklabels=class_names)
    plt.title(f"Confusion Matrix - {fold_idx}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    plt.savefig(SAVE_DIR / f"fold_{fold_idx + 1}_confusion_matrix.png")
    plt.close()

    with open(SAVE_DIR / f"fold_{fold_idx + 1}_predictions.txt", "w") as f:
        for true, pred in zip(label_encoder.inverse_transform(y_val_encoded), label_encoder.inverse_transform(preds_val)):
            f.write(f"{true}\t{pred}\n")

    results.append({"fold": fold_idx + 1, "accuracy": acc, "f1": f1})

# ========== SUMMARY ==========
df_results = pd.DataFrame(results)
print("\nCross-validation results:")
print(df_results)
print("\nMean Accuracy:", df_results["accuracy"].mean())
print("Mean F1 Score:", df_results["f1"].mean())


Total unique trials: 238
Train+Val trials: 215
Test trials: 23

===== Fold 1 =====
Epoch 1: Loss = 2.5807
Epoch 2: Loss = 2.3282
Epoch 3: Loss = 2.2707
Epoch 4: Loss = 2.1522
Epoch 5: Loss = 2.0547
Epoch 6: Loss = 1.9401
Epoch 7: Loss = 1.9316
Epoch 8: Loss = 1.8611
Epoch 9: Loss = 1.8223
Epoch 10: Loss = 1.7084
Epoch 11: Loss = 1.7350
Epoch 12: Loss = 1.6153
Epoch 13: Loss = 1.5035
Epoch 14: Loss = 1.5304
Epoch 15: Loss = 1.5206
Epoch 16: Loss = 1.3805
Epoch 17: Loss = 1.4087
Epoch 18: Loss = 1.3584
Epoch 19: Loss = 1.2612
Epoch 20: Loss = 1.3253
Epoch 21: Loss = 1.2390
Epoch 22: Loss = 1.2172
Epoch 23: Loss = 1.2245
Epoch 24: Loss = 1.1129
Epoch 25: Loss = 1.1231
Epoch 26: Loss = 1.0497
Epoch 27: Loss = 1.0459
Epoch 28: Loss = 1.0587
Epoch 29: Loss = 1.0105
Epoch 30: Loss = 0.9775
Epoch 31: Loss = 0.9297
Epoch 32: Loss = 0.9398
Epoch 33: Loss = 0.9117
Epoch 34: Loss = 0.9705
Epoch 35: Loss = 0.9490
Epoch 36: Loss = 0.8895
Epoch 37: Loss = 0.8100
Epoch 38: Loss = 0.8075
Epoch 39: Loss

In [5]:
# ========== FINAL TEST-SET EVALUATION ==========
import os
import pandas as pd
import numpy as np
import joblib
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix

from src.utils.feature_cleaner   import clean_features
from src.utils.feature_selector  import select_features
from src.models.matching_model   import get_model
from src.utils.label_maps        import TOPIC_TO_META
from src.train.model_configs     import MODEL_CONFIGS, CLASSIC_MODEL_CONFIGS

# ---------------- CONFIG ----------------
TEST_TRIALS_PATH    = f"{PREFIX}notebooks/demo_outputs/test_trials.txt"

BEST_FOLD           = 2
MODEL_WEIGHTS_PATH  = f"{PREFIX}notebooks/demo_outputs/model_fold{BEST_FOLD}.pt"

MODEL_TYPE          = "lightgbm_tuned"   # or "mlp"
FEATURE_GROUPS      = ["rel_theta"]
SLEEP_STAGE_FILTER  = None
EPOCH               = 1
DEVICE              = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------- LOAD TEST TRIALS ----------------
with open(TEST_TRIALS_PATH) as f:
    test_trials = [line.strip() for line in f if line.strip()]

# ---------------- LOAD & CLEAN DATA ----------------
df = pd.read_csv(DATA_PATH)
labels = pd.read_csv(LABEL_PATH)

df["trial_name"]     = df["trial_name"].str.replace(".edf", "", regex=False)
labels["trial_name"] = labels["trial_name"].str.replace(".edf", "", regex=False)
labels["topic_label"] = labels["Meta Topic ID"].map(TOPIC_TO_META)

test_df = df[df["trial_name"].isin(test_trials)]
test_df = test_df.merge(labels[["trial_name", "topic_label"]], on="trial_name", how="left")

test_df = clean_features(test_df)
if isinstance(EPOCH, str) and EPOCH.startswith("avg"):
    test_df = test_df[test_df["epoch"] == EPOCH]
else:
    test_df["epoch"] = pd.to_numeric(test_df["epoch"], errors="coerce")
    test_df = test_df[test_df["epoch"].isin([EPOCH])]


# ---------------- SELECT FEATURES ----------------
X_test, y_test_series = select_features(
    test_df,
    group_names           = FEATURE_GROUPS,
    sleep_stage           = SLEEP_STAGE_FILTER,
    label_df              = None,
    target_column         = "topic_label",
    include_region_summary=False
)

# ---------------- SCALE + ENCODE ----------------
scaler = joblib.load(SCALER_SAVE_PATH)
X_test_scaled = scaler.fit_transform(X_test)

label_encoder = joblib.load(LABEL_ENCODER_PATH)
valid_mask = y_test_series.isin(label_encoder.classes_)
X_test_scaled = X_test_scaled[valid_mask.values]
y_test_enc = label_encoder.transform(y_test_series[valid_mask])

print(f"[Info] Test samples after filtering: {len(y_test_enc)}")

# ---------------- LOAD & PREDICT ----------------
if MODEL_TYPE in CLASSIC_MODEL_CONFIGS:
    config = CLASSIC_MODEL_CONFIGS[MODEL_TYPE]
    model = get_model(model_type=config["model_type"], model_config=config, task_type="classification")
    model.load(MODEL_WEIGHTS_PATH)
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)

else:  # MLP
    config = MODEL_CONFIGS[MODEL_TYPE]
    model = get_model(
        model_type   = MODEL_TYPE,
        model_config = config,
        input_dim    = X_test_scaled.shape[1],
        output_dim   = len(label_encoder.classes_)
    ).to(DEVICE)

    model.load_state_dict(torch.load(MODEL_WEIGHTS_PATH, map_location=DEVICE))
    model.eval()
    with torch.no_grad():
        logits = model(torch.tensor(X_test_scaled, dtype=torch.float32).to(DEVICE))
        y_prob = torch.softmax(logits, dim=1).cpu().numpy()
        y_pred = y_prob.argmax(axis=1)

# ---------------- METRICS ----------------
acc   = accuracy_score(y_test_enc, y_pred)
f1w   = f1_score(y_test_enc, y_pred, average="weighted")
lloss = log_loss(y_test_enc, y_prob, labels=range(len(label_encoder.classes_)))

print("\n───────── TEST RESULTS ─────────")
print(f"Accuracy : {acc:.4f}")
print(f"F1-score : {f1w:.4f}")
print(f"LogLoss  : {lloss:.4f}")

# ---------------- SAVE PREDICTIONS ----------------
test_out_df = pd.DataFrame({
    "trial_name" : test_df["trial_name"].values[valid_mask.values],
    "true_label" : label_encoder.inverse_transform(y_test_enc),
    "pred_label" : label_encoder.inverse_transform(y_pred)
})
test_out_df.to_csv(f"{PREFIX}notebooks/demo_outputs/test_predictions.csv", index=False)
print("[Info] Saved → notebooks/demo_outputs/test_predictions.csv")

# ---------------- CONFUSION MATRIX ----------------
cm = confusion_matrix(y_test_enc, y_pred, labels=range(len(label_encoder.classes_)))
plt.figure(figsize=(12, 12))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Test Confusion Matrix — Topic Classification")
plt.xticks(rotation=17, ha="right")
plt.tight_layout()
plt.savefig(f"{PREFIX}notebooks/demo_outputs/test_confusion_matrix.png")
plt.show()


[Info] Test samples after filtering: 23


FileNotFoundError: [Errno 2] No such file or directory: '/Users/seifelhadidi/Desktop/UNI/Thesis/eeg-dream-decoding-clean/notebooks/demo_outputs/model_fold0.pt'